In [1]:
%matplotlib inline
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import kitti_reader

In [3]:
basedir = '/media/ryan/UBUNTU 16_0/data_odometry_gray/dataset'
sequences = ['00', '01', '02', '03']

In [4]:
from kitti_reader import odometry

In [5]:
dataset = odometry(basedir, sequences[0])

In [6]:
length = len(dataset.timestamps)
print (length)

4541


In [7]:
import itertools

dataset.imformat = 'cv2'
frame = next(iter(dataset.gray))[0]
print (frame.shape)
truth_pose = next(iter(itertools.islice(dataset.poses, 0, None)))
print (truth_pose)
print (frame.shape[0]/3, frame.shape[1]/3)

(376, 1241)
[[  1.00000000e+00   9.04368000e-12   2.32680900e-11   5.55111500e-17]
 [  9.04368300e-12   1.00000000e+00   2.39237000e-10   3.33066900e-16]
 [  2.32681000e-11   2.39237000e-10   9.99999900e-01  -4.44089200e-16]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]
125.33333333333333 413.6666666666667


In [8]:
def data_augment(img):
    img = cv2.resize(img, None, fx=1/3, fy=1/3, interpolation=cv2.INTER_AREA)
    return img

In [9]:
def generator(dataset, batch_size=16):
    batch_frames = []
    batch_truth_pose = []
    while 1:
        for i in range(batch_size):
            frame = next(iter(dataset.gray))[0]
            frame = data_augment(frame)
            frame = np.expand_dims(frame, axis=2)
            batch_frames.append(frame)
            batch_truth_pose.append(next(iter(itertools.islice(dataset.poses, 0, None))))
        yield np.array(batch_frames), np.array(batch_truth_pose)
        

In [10]:
train_generator = generator(dataset)

In [11]:
from keras.layers import *
from keras.models import Model
from keras.preprocessing.image import *
from keras.optimizers import *
from keras.callbacks import *
from keras.applications import *
from keras.losses import *

def build_model(shape):
    rate = 0.2
    inputs = Input(shape=(shape[0], shape[1], 1))
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    pool1 = Dropout(rate)(pool1)
    
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(rate)(pool2)

    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(rate)(pool3)

    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    pool4 = Dropout(rate)(pool4)

    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = Dropout(rate)(conv5)
    
    pose = Conv2D(16, (1, 1), activation=None)(conv5)
    
    model = Model(inputs=[inputs], outputs=[pose])

    model.compile(optimizer=Adam(1e-3), loss=mean_squared_error)
    model.summary()
    return model

shape = [125, 414, 1]
model = build_model(shape)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 125, 414, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 414, 16)      160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 414, 16)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 207, 16)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 207, 16)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 62, 207, 32)       4640      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 62, 207, 32)       9248      
__________

In [12]:
model.fit_generator(train_generator, steps_per_epoch=length)

Epoch 1/1


ValueError: Error when checking target: expected conv2d_11 to have 4 dimensions, but got array with shape (16, 4, 4)